<a href="https://colab.research.google.com/github/akamalas5/Capstone/blob/main/neural_coref.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive/myEnv

/content/drive/My Drive/myEnv


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:27
🔁 Restarting kernel...


In [ ]:
!conda create --name neural_coref
!conda info --envs


Solving environment: | done


==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.11.0

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /usr/local/envs/neural_coref



Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - done
#
# To activate this environment, use
#
#     $ conda activate neural_coref
#
# To deactivate an active environment, use
#
#     $ conda deactivate

# conda environments:
#
base                  *  /usr/local
neural_coref             /usr/local/envs/neural_coref



In [ ]:
!conda update -n base conda


Solving environment: - \ | / - \ | / - \ | / - \ | / - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    brotlipy-0.7.0             |py37h5e8e339_1003         342 KB  conda-forge
    c-ares-1.18.1              |       h7f98852_0         113 KB  conda-forge
    ca-certificates-2021.10.8  |       ha878542_0         139 KB  conda-forge
    certifi-2021.10.8          |   py37h89c1867_1         145 KB  conda-forge
    cffi-1.15.0                |   py37h036bc23_0         225 KB  conda-forge
    charset-normalizer-2.0.8   |     pyhd8ed1ab_0          34 KB  conda-forge
    colorama-0.4.4             |     pyh9f0ad1d_0          18 KB  conda-forge
    conda-4.11.0               |   py37h89c1867_0        16.9 MB  conda-forge
    conda-package-handling-1.7.3|   py

In [ ]:
!conda init bash

no change     /usr/local/condabin/conda
no change     /usr/local/bin/conda
no change     /usr/local/bin/conda-env
no change     /usr/local/bin/activate
no change     /usr/local/bin/deactivate
no change     /usr/local/etc/profile.d/conda.sh
no change     /usr/local/etc/fish/conf.d/conda.fish
no change     /usr/local/shell/condabin/Conda.psm1
no change     /usr/local/shell/condabin/conda-hook.ps1
no change     /usr/local/lib/python3.7/site-packages/xontrib/conda.xsh
no change     /usr/local/etc/profile.d/conda.csh
modified      /root/.bashrc

==> For changes to take effect, close and re-open your current shell. <==



In [ ]:
!conda activate neural_coref



CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




In [ ]:
!apt-get install python3.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3.7 is already the newest version (3.7.12-1+bionic1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
!pip install spacy==2.1.0
!pip install neuralcoref
!pip install https://github.com/explosion/spacy-models/releases//download/en_core_web_lg-2.1.0/en_core_web_lg-2.1.0.tar.gz

     |████████████████████████████████| 27.7 MB 97.2 MB/s 
     |████████████████████████████████| 3.2 MB 59.1 MB/s 
     |████████████████████████████████| 15.7 MB 61.9 MB/s 
     |████████████████████████████████| 184 kB 76.7 MB/s 
     |████████████████████████████████| 82 kB 522 kB/s 
     |████████████████████████████████| 2.1 MB 71.6 MB/s 


     |████████████████████████████████| 286 kB 3.9 MB/s 
     |████████████████████████████████| 131 kB 72.0 MB/s 
     |████████████████████████████████| 8.2 MB 57.0 MB/s 
     |████████████████████████████████| 79 kB 10.5 MB/s 
     |████████████████████████████████| 247 kB 106.3 MB/s 


     |████████████████████████████████| 826.9 MB 20 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.1.0-py3-none-any.whl size=828255075 sha256=e719b6233f3f48d5a729843e6e2b7b109e174e2dd4d475bf2176a96e474d74b4
  Stored in directory: /root/.cache/pip/wheels/26/0b/83/1848f0f2bf9022e2195aea917615e61573a70367c21b5ae4b7
Successfully built en-core-web-lg


In [ ]:
import pandas as pd
import re
import spacy
import neuralcoref
import en_core_web_lg

nlp = en_core_web_lg.load()
neuralcoref.add_to_pipe(nlp)

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: spacy.tokens.span.Span size changed, may indicate binary incompatibility. Expected 72 from C header, got 80 from PyObject
  return f(*args, **kwds)
100%|██████████| 40155833/40155833 [00:04<00:00, 8185624.02B/s] 


In [ ]:
cd /content/drive/My Drive/myEnv

/content/drive/My Drive/myEnv


In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
import wikipedia
from collections import Counter
import matplotlib.pyplot as plt
import networkx as nx
from tqdm import tqdm
import pandas as pd
import numpy as np
from collections import defaultdict

In [ ]:
result = wikipedia.search("Braking System")
result

['Anti-lock braking system',
 'Brake',
 'Combined braking system',
 'Collision avoidance system',
 'Regenerative brake',
 'Bicycle brake',
 'Brake-by-wire',
 'Emergency brake assist',
 'Railway air brake',
 'List of aircraft braking systems']

In [ ]:
wiki_url = 'https://en.wikipedia.org/wiki/braking_system' # 'https://en.wikipedia.org/wiki/Disc_brake'
print('Fetching main wiki article: %s' % wiki_url)
page = requests.get(wiki_url)
print('Done. Extracting table links..')
html = BeautifulSoup(page.text)
# table = html.find('table', 'wikitable')
# links = table.findAll('a')
sents = html.find_all('p')
links = html.find_all('a')

print('Done extracting links. About to fetch: %s links..' % len(links))

Fetching main wiki article: https://en.wikipedia.org/wiki/braking_system
Done. Extracting table links..
Done extracting links. About to fetch: 565 links..


In [ ]:
 sents[0].text

'A brake is a mechanical device that inhibits motion by absorbing energy from a moving system.[1] It is used for slowing or stopping a moving vehicle, wheel, axle, or to prevent its motion, most often accomplished by means of friction.[2]\n'

In [ ]:
for sent in sents[1:2]:
    doc = nlp(sent.text)

    for token in doc:
        # extract subject
        if (token.dep_=='nsubj'):
            print("Subj: ",token.text)
        # extract object
        elif (token.dep_=='dobj'):
            print("Obj: ",token.text)
        elif(token.pos_=="VERB"):
            print("Verb: ", token.text)
    print("======================================")

Subj:  brakes
Verb:  use
Obj:  friction
Verb:  pressed
Verb:  convert
Obj:  energy
Verb:  moving
Verb:  may
Verb:  be
Verb:  employed
Subj:  braking
Verb:  converts
Obj:  much
Verb:  may
Verb:  be
Verb:  stored
Subj:  methods
Verb:  convert
Obj:  energy
Verb:  stored
Verb:  pressurized
Verb:  pressurized
Subj:  brakes
Verb:  use
Obj:  fields
Verb:  convert
Obj:  energy
Verb:  is
Verb:  converted
Subj:  methods
Verb:  transform
Obj:  energy
Verb:  transferring
Obj:  energy


In [ ]:
wiki_text = defaultdict(list)
for sent in sents:
    if(len(sent.text)>1):
        wiki_text['rqt_text'].append(sent.text)

rqt_df = pd.DataFrame(wiki_text)
rqt_df.head(), len(rqt_df)

(                                            rqt_text
 0  A brake is a mechanical device that inhibits m...
 1  Most brakes commonly use friction between two ...
 2  Brakes are generally applied to rotating axles...
 3  Since kinetic energy increases quadratically w...
 4  Almost all wheeled vehicles have a brake of so..., 35)

In [ ]:
# function to preprocess rqt_text
def clean(text):
    
    # removing paragraph numbers
    text = re.sub('[0-9]+.','',str(text))
    
    # removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    
    # removing hyphens
    text = re.sub("-",' ',str(text))
    text = re.sub("— ",'',str(text))
    # removing quotation marks
    text = re.sub('\"','',str(text))

    # removing any reference to outside text
    text = re.sub("[\(\[.*?[\)\]]", "", str(text))
    
    # removing table tags
    text = re.sub('&nbsp;', ' ', str(text))

    # removing table tags
    text = re.sub('  ', ' ', str(text))
    
    return text

# preprocessing req_text
rqt_df['text_clean'] = rqt_df['rqt_text'].apply(clean)

In [ ]:
def coref_resolution(text):
    """Function that executes coreference resolution on a given text"""
    doc = nlp(text)
    # fetches tokens with whitespaces from spacy document
    tok_list = list(token.text_with_ws for token in doc)
    for cluster in doc._.coref_clusters:
        # get tokens from representative cluster name
        cluster_main_words = set(cluster.main.text.split(' '))
        for coref in cluster:
            if coref != cluster.main:  # if coreference element is not the representative element of that cluster
                if coref.text != cluster.main.text and bool(set(coref.text.split(' ')).intersection(cluster_main_words)) == False:
                    # if coreference element text and representative element text are not equal and none of the coreference element words are in representative element. This was done to handle nested coreference scenarios
                    tok_list[coref.start] = cluster.main.text + \
                        doc[coref.end-1].whitespace_
                    for i in range(coref.start+1, coref.end):
                        tok_list[i] = ""

    return "".join(tok_list)

In [ ]:
cd /content/drive/My Drive/myEnv

/content/drive/My Drive/myEnv


In [ ]:
coref_resolution('The brake system and hose system should to be mounted on chassis.  it should also be linked with pedal. Pedel is part of under body. it should be in the hood') 

'The brake system and hose system should to be mounted on chassis.  The brake system and hose system should also be linked with pedal. Pedel is part of under body. Pedel should be in the hood'

In [ ]:
# input_data = "/content/drive/My Drive/Capstone/data/wiki_brake_all_pages_df.csv"
# input_data = "/content/drive/My Drive/Capstone/data/wiki_brake_full_coref_df.csv"
# input_data = "/content/drive/My Drive/Capstone/data/reqt_entities_df.csv"
input_data = "/content/drive/My Drive/Capstone/data/req_brake_coref_df.xlsx"


In [ ]:
cd ../Capstone/data/req_brake_coref_df.xlsx

'/content/drive/My Drive/myEnv'

In [ ]:
brake_df = pd.read_excel(input_data)
brake_df.head()

,No,id,Type,date,rqt_text,text_clean
0,0,REG-001200-007515/8,RegReqment,2020-11-09 19:44:00,REG-001200-007515&#x00a0;&#x00a0;&#x00a0;&#x00...,REG #x&#x&#x&#x&#x&#x&#x&#xVB nbsp;Title: USA/...
1,1,REG-001200-008241/6,RegReqment,2021-02-05 16:16:00,Regulation Summary: C/ FMVSS 136 Electronic...,Regulation Summary: C/ FMVSS Electronic Stabil...
2,2,REG-001200-008317/2,RegReqment,2016-12-22 14:11:00,The National Traffic Department - DENATRAN iss...,The National Traffic Department DENATRAN issu...
3,3,REG-001200-008587/3,RegReqment,2018-01-24 15:46:00,REG-001200-008587ROK ESC requirement&nbsp;Arti...,REG OK ESC requirement Article Electronic Stab...
4,4,REG-001204-008553/5,RegReqment,2020-12-11 05:26:00,REG- 001204 - 008553 CHN General Requireme...,REG CHN General Requirements for Brake Syste...


In [ ]:
brake_df['text_clean'] = brake_df['text_clean'].astype(str)
brake_df['tex_coref_resolved'] = brake_df['text_clean'].apply(coref_resolution)

In [ ]:
brake_df.head()

,No,id,Type,date,rqt_text,text_clean,tex_coref_resolved
0,0,REG-001200-007515/8,RegReqment,2020-11-09 19:44:00,REG-001200-007515&#x00a0;&#x00a0;&#x00a0;&#x00...,REG #x&#x&#x&#x&#x&#x&#x&#xVB nbsp;Title: USA/...,REG #x&#x&#x&#x&#x&#x&#x&#xVB nbsp;Title: USA/...
1,1,REG-001200-008241/6,RegReqment,2021-02-05 16:16:00,Regulation Summary: C/ FMVSS 136 Electronic...,Regulation Summary: C/ FMVSS Electronic Stabil...,Regulation Summary: C/ FMVSS Electronic Stabil...
2,2,REG-001200-008317/2,RegReqment,2016-12-22 14:11:00,The National Traffic Department - DENATRAN iss...,The National Traffic Department DENATRAN issu...,The National Traffic Department DENATRAN issu...
3,3,REG-001200-008587/3,RegReqment,2018-01-24 15:46:00,REG-001200-008587ROK ESC requirement&nbsp;Arti...,REG OK ESC requirement Article Electronic Stab...,REG OK ESC requirement Article Electronic Stab...
4,4,REG-001204-008553/5,RegReqment,2020-12-11 05:26:00,REG- 001204 - 008553 CHN General Requireme...,REG CHN General Requirements for Brake Syste...,REG CHN General Requirements for Brake Syste...


In [ ]:
brake_df.to_csv("/content/drive/My Drive/Capstone/data/req_brakeswith_coref_df.csv")